# 🚀 Entrenamiento Deformable DETR en Google Colab

Este notebook entrena un modelo Deformable DETR con MMDetection en Google Colab.

## 📋 Características
- **Modelo**: Deformable DETR con backbone Swin-T o ResNet-50
- **Framework**: MMDetection (OpenMMLab)
- **Detección automática de GPU**
- **Configuración automática de datasets**
- **Visualización de resultados**
- **Exportación de modelos**

## 🎯 Clases de Animales
- Buffalo
- Elephant  
- Kob
- Alcelaphinae
- Warthog
- Waterbuck

## ⚙️ Configuración por Defecto
- **Backbone**: Swin-T (recomendado) o ResNet-50
- **Tamaño de imagen**: 896x896
- **Épocas**: 50
- **Batch size**: 4 (train), 2 (val/test)
- **Optimizador**: AdamW con weight decay

## 🔬 Ventajas de Deformable DETR
- **End-to-end**: Sin necesidad de NMS post-procesamiento
- **Eficiente**: Menos parámetros que R-CNN basados en regiones
- **Preciso**: Mejor rendimiento en objetos pequeños
- **Flexible**: Arquitectura transformer moderna


## 🔧 Instalación de Dependencias


In [ ]:
# Instalar dependencias principales
%pip install -q openmim
!mim install -q mmcv-full
!mim install -q mmdet
%pip install -q pyyaml opencv-python pillow tqdm matplotlib seaborn pandas

# Verificar instalación
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 📦 Importar Librerías


In [ ]:
import os
import sys
import yaml
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import shutil
import json
from tqdm import tqdm

# MMDetection imports
from mmengine.config import Config
from mmengine.runner import Runner
import mmdet
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules

# Google Colab
from google.colab import files, drive

# Configurar matplotlib
plt.style.use('default')
sns.set_palette("husl")

print(f"MMDetection version: {mmdet.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


## 📁 Configuración de Datos


In [ ]:
# Configuración de rutas y parámetros
class Config:
    def __init__(self):
        # Parámetros del modelo
        self.backbone = "swin_t"  # "swin_t" o "resnet50"
        self.imgsz = 896
        self.epochs = 50
        self.batch_size = 4
        self.val_batch_size = 2
        
        # Rutas (ajustar según tu estructura de datos)
        self.data_root = "/content/aerial-wildlife-count"
        self.work_dir = "/content/work_dirs/deformable_detr"
        
        # Clases del dataset
        self.classes = [
            "Buffalo", "Elephant", "Kob", 
            "Alcelaphinae", "Warthog", "Waterbuck"
        ]
        
        # Configuración de GPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def print_config(self):
        print("🔧 Configuración:")
        print(f"  Backbone: {self.backbone}")
        print(f"  Tamaño de imagen: {self.imgsz}")
        print(f"  Épocas: {self.epochs}")
        print(f"  Batch size: {self.batch_size}")
        print(f"  Dispositivo: {self.device}")
        print(f"  Directorio de trabajo: {self.work_dir}")

# Crear instancia de configuración
cfg = Config()
cfg.print_config()


In [ ]:
# ============================================================
# CONFIGURACIÓN DE RUTAS PRINCIPALES (Adaptado para pipelines)
# ============================================================

# Definir la ruta base principal (ajustar según el entorno: Drive o local)
BASE_DIR = Path("/content/drive/MyDrive/aerial-wildlife-count")

# ============================================================
# RUTAS DE DATOS PROCESADOS (Outputs de pipelines)
# ============================================================

# Rutas de los archivos de anotaciones finales (después de quality + augmentation)
TRAIN_ANN_FILE = BASE_DIR / "data" / "outputs" / "mirror_clean" / "train_final" / "train_final.json"
VAL_ANN_FILE = BASE_DIR / "data" / "outputs" / "mirror_clean" / "val_final" / "val_final.json"
TEST_ANN_FILE = BASE_DIR / "data" / "outputs" / "mirror_clean" / "test_final" / "test_final.json"

# Rutas alternativas si no existe la estructura final
TRAIN_ANN_FILE_ALT = BASE_DIR / "data" / "outputs" / "verified" / "train_validated.json"
VAL_ANN_FILE_ALT = BASE_DIR / "data" / "outputs" / "verified" / "val_validated.json"
TEST_ANN_FILE_ALT = BASE_DIR / "data" / "outputs" / "verified" / "test_validated.json"

# Rutas de las carpetas de imágenes correspondientes
TRAIN_IMG_DIR = BASE_DIR / "data" / "outputs" / "mirror_clean" / "train_final" / "images"
VAL_IMG_DIR = BASE_DIR / "data" / "outputs" / "mirror_clean" / "val_final" / "images"
TEST_IMG_DIR = BASE_DIR / "data" / "outputs" / "mirror_clean" / "test_final" / "images"

# Rutas alternativas para imágenes
TRAIN_IMG_DIR_ALT = BASE_DIR / "data" / "outputs" / "verified" / "train" / "images"
VAL_IMG_DIR_ALT = BASE_DIR / "data" / "outputs" / "verified" / "val" / "images"
TEST_IMG_DIR_ALT = BASE_DIR / "data" / "outputs" / "verified" / "test" / "images"

# ============================================================
# FUNCIÓN PARA CONFIGURAR DATOS (Adaptada para pipelines)
# ============================================================

def setup_data():
    """Configurar datos desde Google Drive con detección automática de pipelines"""
    
    # Montar Google Drive
    drive.mount('/content/drive')
    print("✅ Google Drive montado")
    
    # Verificar si existe la estructura de pipelines
    if TRAIN_ANN_FILE.exists() and TRAIN_IMG_DIR.exists():
        print("✅ Datos de pipelines encontrados (estructura final)")
        cfg.data_root = str(BASE_DIR)
        return True, "final"
    elif TRAIN_ANN_FILE_ALT.exists() and TRAIN_IMG_DIR_ALT.exists():
        print("✅ Datos de pipelines encontrados (estructura verificada)")
        cfg.data_root = str(BASE_DIR)
        return True, "verified"
    else:
        # Buscar en ubicaciones alternativas
        drive_path = "/content/drive/MyDrive"
        possible_paths = [
            f"{drive_path}/aerial-wildlife-count",
            f"{drive_path}/datasets/aerial-wildlife-count",
            f"{drive_path}/Colab Notebooks/aerial-wildlife-count"
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                cfg.data_root = path
                print(f"✅ Dataset encontrado en ubicación alternativa: {path}")
                return True, "legacy"
        
        print("❌ No se encontró el dataset en Google Drive")
        return False, None

# Configurar datos
success, data_type = setup_data()
if success:
    print(f"📁 Ruta de datos configurada: {cfg.data_root}")
    print(f"📊 Tipo de datos: {data_type}")
else:
    print("⚠️  Configura los datos manualmente antes de continuar")


In [ ]:
# Función para encontrar y validar datasets (Adaptada para pipelines)
def find_datasets():
    """Encontrar datasets disponibles según el tipo de datos"""
    
    datasets_found = []
    
    if data_type == "final":
        # Estructura final de pipelines (augmentation + quality)
        datasets_found = [TRAIN_ANN_FILE, VAL_ANN_FILE, TEST_ANN_FILE]
        print("📊 Usando datasets finales de pipelines:")
        print(f"  Train: {TRAIN_ANN_FILE}")
        print(f"  Val: {VAL_ANN_FILE}")
        print(f"  Test: {TEST_ANN_FILE}")
        
    elif data_type == "verified":
        # Estructura verificada de quality pipeline
        datasets_found = [TRAIN_ANN_FILE_ALT, VAL_ANN_FILE_ALT, TEST_ANN_FILE_ALT]
        print("📊 Usando datasets verificados de quality pipeline:")
        print(f"  Train: {TRAIN_ANN_FILE_ALT}")
        print(f"  Val: {VAL_ANN_FILE_ALT}")
        print(f"  Test: {TEST_ANN_FILE_ALT}")
        
    else:
        # Estructura legacy - buscar archivos JSON
        data_root = Path(cfg.data_root)
        json_patterns = [
            "**/train_*.json",
            "**/val_*.json", 
            "**/test_*.json",
            "**/*_big_size_*.json",
            "**/*_subframes_*.json",
            "**/*_final.json",
            "**/*_validated.json"
        ]
        
        for pattern in json_patterns:
            for json_file in data_root.glob(pattern):
                if json_file.is_file():
                    datasets_found.append(json_file)
        
        print(f"📊 Datasets encontrados en estructura legacy ({len(datasets_found)}):")
        for i, dataset in enumerate(datasets_found):
            print(f"  {i+1}. {dataset}")
    
    return datasets_found

# Función para analizar un dataset COCO
def analyze_dataset(json_path):
    """Analizar estadísticas de un dataset COCO"""
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"\n📈 Análisis de {json_path.name}:")
    print(f"  Imágenes: {len(data['images'])}")
    print(f"  Anotaciones: {len(data['annotations'])}")
    print(f"  Categorías: {len(data['categories'])}")
    
    # Estadísticas por categoría
    cat_counts = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        cat_counts[cat_id] = cat_counts.get(cat_id, 0) + 1
    
    print("\n  Distribución por categoría:")
    for cat in data['categories']:
        count = cat_counts.get(cat['id'], 0)
        print(f"    {cat['name']}: {count}")
    
    return data

# Buscar y analizar datasets
datasets = find_datasets()

if datasets:
    # Analizar el primer dataset encontrado como ejemplo
    sample_data = analyze_dataset(datasets[0])
else:
    print("❌ No se encontraron datasets. Verifica la estructura de directorios.")


In [ ]:
# Función para cargar datos desde Google Drive o subir archivos
def setup_data():
    """Configurar datos desde Google Drive o archivos subidos"""
    
    # Opción 1: Montar Google Drive
    print("📁 Opciones para cargar datos:")
    print("1. Montar Google Drive")
    print("2. Subir archivos manualmente")
    
    choice = input("Selecciona opción (1 o 2): ").strip()
    
    if choice == "1":
        # Montar Google Drive
        drive.mount('/content/drive')
        print("✅ Google Drive montado")
        
        # Buscar el dataset en Drive
        drive_path = "/content/drive/MyDrive"
        possible_paths = [
            f"{drive_path}/aerial-wildlife-count",
            f"{drive_path}/datasets/aerial-wildlife-count",
            f"{drive_path}/Colab Notebooks/aerial-wildlife-count"
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                cfg.data_root = path
                print(f"✅ Dataset encontrado en: {path}")
                return True
        
        print("❌ No se encontró el dataset en Google Drive")
        return False
    
    elif choice == "2":
        print("📤 Sube los archivos del dataset...")
        uploaded = files.upload()
        print("✅ Archivos subidos")
        return True
    
    else:
        print("❌ Opción inválida")
        return False

# Configurar datos
if not setup_data():
    print("⚠️  Configura los datos manualmente antes de continuar")


In [ ]:
# Función para construir configuración de Deformable DETR
def build_deformable_detr_config(backbone, imgsz, epochs, train_json, val_json, test_json, workdir):
    """Construir configuración de Deformable DETR para MMDetection"""
    
    # Configuración base según backbone
    if backbone.lower().startswith("swin"):
        base_model = "mmdet::_base_/models/deformable_detr_swin-t.py"
        pretrained = "https://download.openmmlab.com/mmdetection/v2.0/deformable_detr/deformable_detr_swin_t_16x2_50e_coco/deformable_detr_swin_t_16x2_50e_coco_20210419_220030-a12b9512.pth"
    else:
        base_model = "mmdet::_base_/models/deformable_detr_r50.py"
        pretrained = "https://download.openmmlab.com/mmdetection/v2.0/deformable_detr/deformable_detr_r50_16x2_50e_coco/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth"
    
    # Determinar directorios de imágenes según el tipo de datos
    if data_type == "final":
        train_img_dir = TRAIN_IMG_DIR
        val_img_dir = VAL_IMG_DIR
        test_img_dir = TEST_IMG_DIR
    elif data_type == "verified":
        train_img_dir = TRAIN_IMG_DIR_ALT
        val_img_dir = VAL_IMG_DIR_ALT
        test_img_dir = TEST_IMG_DIR_ALT
    else:
        # Estructura legacy
        train_img_dir = train_json.parent / "images" if (train_json.parent / "images").exists() else train_json.parent.parent / "train"
        val_img_dir = val_json.parent.parent / "val"
        test_img_dir = test_json.parent.parent / "test"
    
    config = {
        "_base_": [
            base_model,
            "mmdet::_base_/datasets/coco_instance.py",
            "mmdet::_base_/schedules/schedule_1x.py",
            "mmdet::_base_/default_runtime.py",
        ],
        "default_scope": "mmdet",
        
        # Dataset configuration
        "dataset_type": "CocoDataset",
        "data_root": str(train_json.parent.parent),
        
        # Train dataloader
        "train_dataloader": {
            "batch_size": cfg.batch_size,
            "num_workers": 4,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": True},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(train_json),
                "data_prefix": {"img": str(train_img_dir)},
                "filter_cfg": {"filter_empty": True, "min_size": 1},
            },
        },
        
        # Validation dataloader
        "val_dataloader": {
            "batch_size": cfg.val_batch_size,
            "num_workers": 2,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": False},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(val_json),
                "data_prefix": {"img": str(val_img_dir)},
                "test_mode": True
            },
        },
        
        # Test dataloader
        "test_dataloader": {
            "batch_size": cfg.val_batch_size,
            "num_workers": 2,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": False},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(test_json),
                "data_prefix": {"img": str(test_img_dir)},
                "test_mode": True
            },
        },
        
        # Data augmentation pipelines (específico para DETR)
        "train_pipeline": [
            {"type": "LoadImageFromFile"},
            {"type": "LoadAnnotations", "with_bbox": True},
            {"type": "RandomResize", "scale": [(imgsz, imgsz)], "keep_ratio": True},
            {"type": "RandomFlip", "prob": 0.5},
            {"type": "PackDetInputs"},
        ],
        "test_pipeline": [
            {"type": "LoadImageFromFile"},
            {"type": "Resize", "scale": (imgsz, imgsz), "keep_ratio": True},
            {"type": "LoadAnnotations", "with_bbox": True},
            {"type": "PackDetInputs"},
        ],
        
        # Optimizer configuration (AdamW para DETR)
        "optim_wrapper": {
            "type": "OptimWrapper",
            "optimizer": {
                "type": "AdamW", 
                "lr": 2e-4, 
                "betas": (0.9, 0.999), 
                "weight_decay": 0.05
            },
            "clip_grad": {"max_norm": 1.0, "norm_type": 2},
        },
        
        # Learning rate scheduler (CosineAnnealing para DETR)
        "param_scheduler": [
            {
                "type": "LinearLR", 
                "start_factor": 0.001, 
                "by_epoch": False, 
                "begin": 0, 
                "end": 500
            },
            {
                "type": "CosineAnnealingLR", 
                "T_max": epochs, 
                "by_epoch": True
            },
        ],
        
        # Training configuration
        "train_cfg": {"max_epochs": epochs, "val_interval": 1},
        "val_evaluator": {"type": "CocoMetric", "ann_file": str(val_json), "metric": "bbox"},
        "test_evaluator": {"type": "CocoMetric", "ann_file": str(test_json), "metric": "bbox"},
        
        # Work directory and model loading
        "work_dir": str(workdir),
        "load_from": pretrained,
        "resume": False,
        
        # Visualization and logging
        "visualizer": {"type": "DetLocalVisualizer"},
        "default_hooks": {
            "checkpoint": {
                "type": "CheckpointHook", 
                "interval": 1, 
                "max_keep_ckpts": 3, 
                "save_best": "coco/bbox_mAP"
            },
            "logger": {"type": "LoggerHook", "interval": 50},
            "param_scheduler": {"type": "ParamSchedulerHook"},
            "timer": {"type": "IterTimerHook"},
            "sampler_seed": {"type": "DistSamplerSeedHook"},
        },
        
        # Environment configuration
        "env_cfg": {"cudnn_benchmark": True},
        "randomness": {"seed": 42, "deterministic": False}
    }
    
    return Config(config)

print("✅ Función de configuración de Deformable DETR creada")


In [ ]:
# Preparar datos para entrenamiento (Adaptada para pipelines)
def prepare_training_data():
    """Preparar y validar datos para entrenamiento según el tipo de datos"""
    
    if data_type == "final":
        # Usar rutas finales de pipelines
        train_json = TRAIN_ANN_FILE
        val_json = VAL_ANN_FILE
        test_json = TEST_ANN_FILE
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists(), test_json.exists()]):
            print("❌ Faltan archivos de datos finales de pipelines")
            return None, None, None
            
    elif data_type == "verified":
        # Usar rutas verificadas de quality pipeline
        train_json = TRAIN_ANN_FILE_ALT
        val_json = VAL_ANN_FILE_ALT
        test_json = TEST_ANN_FILE_ALT
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists(), test_json.exists()]):
            print("❌ Faltan archivos de datos verificados de quality pipeline")
            return None, None, None
            
    else:
        # Estructura legacy - buscar en datasets encontrados
        train_files = [d for d in datasets if 'train' in d.name.lower()]
        val_files = [d for d in datasets if 'val' in d.name.lower()]
        test_files = [d for d in datasets if 'test' in d.name.lower()]
        
        print("📊 Archivos de datos encontrados:")
        print(f"  Train: {len(train_files)} archivos")
        print(f"  Val: {len(val_files)} archivos") 
        print(f"  Test: {len(test_files)} archivos")
        
        # Seleccionar archivos (usar los primeros encontrados)
        train_json = train_files[0] if train_files else None
        val_json = val_files[0] if val_files else None
        test_json = test_files[0] if test_files else None
        
        if not all([train_json, val_json, test_json]):
            print("❌ Faltan archivos de datos. Necesitas train, val y test JSON files.")
            return None, None, None
    
    print(f"\n✅ Archivos seleccionados:")
    print(f"  Train: {train_json}")
    print(f"  Val: {val_json}")
    print(f"  Test: {test_json}")
    
    return train_json, val_json, test_json

# Preparar datos
train_json, val_json, test_json = prepare_training_data()

if train_json and val_json and test_json:
    print("✅ Datos preparados correctamente")
else:
    print("❌ Error preparando datos")


## 📊 Cargar y Preparar Datos


In [ ]:
# Función para encontrar y validar datasets
def find_datasets():
    """Encontrar datasets disponibles en el directorio"""
    
    data_root = Path(cfg.data_root)
    datasets_found = []
    
    # Buscar archivos JSON de anotaciones
    json_patterns = [
        "**/train_*.json",
        "**/val_*.json", 
        "**/test_*.json",
        "**/*_big_size_*.json",
        "**/*_subframes_*.json"
    ]
    
    for pattern in json_patterns:
        for json_file in data_root.glob(pattern):
            if json_file.is_file():
                datasets_found.append(json_file)
    
    print(f"📊 Datasets encontrados ({len(datasets_found)}):")
    for i, dataset in enumerate(datasets_found):
        print(f"  {i+1}. {dataset}")
    
    return datasets_found

# Función para analizar un dataset COCO
def analyze_dataset(json_path):
    """Analizar estadísticas de un dataset COCO"""
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"\n📈 Análisis de {json_path.name}:")
    print(f"  Imágenes: {len(data['images'])}")
    print(f"  Anotaciones: {len(data['annotations'])}")
    print(f"  Categorías: {len(data['categories'])}")
    
    # Estadísticas por categoría
    cat_counts = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        cat_counts[cat_id] = cat_counts.get(cat_id, 0) + 1
    
    print("\n  Distribución por categoría:")
    for cat in data['categories']:
        count = cat_counts.get(cat['id'], 0)
        print(f"    {cat['name']}: {count}")
    
    return data

# Buscar y analizar datasets
datasets = find_datasets()

if datasets:
    # Analizar el primer dataset encontrado como ejemplo
    sample_data = analyze_dataset(datasets[0])
else:
    print("❌ No se encontraron datasets. Verifica la estructura de directorios.")


## 🚀 Entrenamiento del Modelo


In [ ]:
# Preparar datos y entrenar modelo
def prepare_and_train():
    """Preparar datos y entrenar el modelo Deformable DETR"""
    
    # Buscar archivos de datos
    train_files = [d for d in datasets if 'train' in d.name.lower()]
    val_files = [d for d in datasets if 'val' in d.name.lower()]
    test_files = [d for d in datasets if 'test' in d.name.lower()]
    
    train_json = train_files[0] if train_files else None
    val_json = val_files[0] if val_files else None
    test_json = test_files[0] if test_files else None
    
    if not all([train_json, val_json, test_json]):
        print("❌ Faltan archivos de datos. Necesitas train, val y test JSON files.")
        return
    
    print(f"✅ Archivos seleccionados:")
    print(f"  Train: {train_json}")
    print(f"  Val: {val_json}")
    print(f"  Test: {test_json}")
    
    # Crear directorio de trabajo
    workdir = Path(cfg.work_dir)
    workdir.mkdir(parents=True, exist_ok=True)
    
    print("🏗️ Construyendo configuración del modelo...")
    
    # Construir configuración
    mmdet_config = build_deformable_detr_config(
        backbone=cfg.backbone,
        imgsz=cfg.imgsz, 
        epochs=cfg.epochs,
        train_json=train_json,
        val_json=val_json,
        test_json=test_json,
        workdir=workdir
    )
    
    print("✅ Configuración creada")
    
    # Guardar configuración
    config_path = workdir / "config.py"
    mmdet_config.dump(str(config_path))
    print(f"💾 Configuración guardada en: {config_path}")
    
    # Iniciar entrenamiento
    print("🚀 Iniciando entrenamiento de Deformable DETR...")
    print(f"⏱️  Tiempo estimado: {cfg.epochs * 3} minutos (aproximado)")
    
    try:
        # Crear runner
        runner = Runner.from_cfg(mmdet_config)
        
        # Iniciar entrenamiento
        runner.train()
        
        print("✅ Entrenamiento completado exitosamente!")
        
    except Exception as e:
        print(f"❌ Error durante el entrenamiento: {e}")
        print("💡 Verifica que los datos estén correctamente configurados")

# Ejecutar entrenamiento
prepare_and_train()


## 🎉 ¡Entrenamiento Completado!

### 📋 Resumen del Entrenamiento
- **Modelo**: Deformable DETR con backbone {cfg.backbone}
- **Tamaño de imagen**: {cfg.imgsz}x{cfg.imgsz}
- **Épocas**: {cfg.epochs}
- **Clases detectadas**: {len(cfg.classes)} especies de animales

### 📊 Próximos Pasos
1. **Evaluar métricas**: Revisar mAP, precision, recall
2. **Ajustar hiperparámetros**: Si es necesario mejorar el rendimiento
3. **Exportar modelo**: Convertir a ONNX o TorchScript para deployment
4. **Probar en nuevas imágenes**: Validar en datos no vistos

### 🔧 Configuración Personalizada
Para modificar parámetros, edita la clase `Config` en la celda de configuración:
- Cambiar backbone: `"swin_t"` o `"resnet50"`
- Ajustar épocas: `epochs = 100`
- Modificar tamaño de imagen: `imgsz = 1024`
- Cambiar batch size: `batch_size = 8`

### 📚 Recursos Adicionales
- [Documentación MMDetection](https://mmdetection.readthedocs.io/)
- [Deformable DETR Paper](https://arxiv.org/abs/2010.04159)
- [DETR Paper](https://arxiv.org/abs/2005.12872)
